### Расчет индекса вовлеченности групп https://vk.com/habr и https://vk.com/popularmechanics

In [1]:
# Это отношение количества подписчиков группы к сумме комментариев, лайков и репостов всех записей за определенный период
# Чем этот показатель меньше, тем более активно подписчики группы реагируют на публикуемый в ней контент.

import requests
from yaml import safe_load
f = open('config.yaml', mode = 'r', encoding = 'utf-8')
config = safe_load(f)
token = config['access_token']

# 100 постов группы хабр
url = 'https://api.vk.com/method/wall.get'

params = {
    'domain': 'habr',
    'filter': 'owner',
    'count': 100,
    'offset': 0,
    'access_token': token,
    'v': 5.73
}
data_habr = requests.get(url, params = params)

# 100 постов группы механика
url = 'https://api.vk.com/method/wall.get'

params = {
    'domain': 'popularmechanics',
    'filter': 'owner',
    'count': 100,
    'offset': 0,
    'access_token': token,
    'v': 5.73
}
data_mech = requests.get(url, params = params)

In [2]:
# Собираем статистику постов хабра
Started = True
stats_habr = {}
for record in data_habr.json()['response']['items']:
    if Started:
        Started = False    
    else:
        if 'attachments' in record:
            if 'link' in record['attachments'][0]:
                title = record['attachments'][0]['link']['title']
            elif 'photo' in record['attachments'][0]:
                title = record['attachments'][0]['photo']['text']
        elif 'copy_history' in record:
            if 'link' in record['copy_history'][0]['attachments'][0]:
                title = record['copy_history'][0]['attachments'][0]['link']['title']
            elif 'poll' in record['copy_history'][0]['attachments'][0]:
                title = record['copy_history'][0]['attachments'][0]['poll']['question']
        stats_habr[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date']]
        
# Собираем статистику постов механики
Started = True
stats_mech = {}
for record in data_mech.json()['response']['items']:
    if Started:
        Started = False    
    else:
        if 'attachments' in record:
            if 'link' in record['attachments'][0]:
                title = record['attachments'][0]['link']['title']
            elif 'photo' in record['attachments'][0]:
                title = record['attachments'][0]['photo']['text']
        elif 'copy_history' in record:
            if 'link' in record['copy_history'][0]['attachments'][0]:
                title = record['copy_history'][0]['attachments'][0]['link']['title']
            elif 'poll' in record['copy_history'][0]['attachments'][0]:
                title = record['copy_history'][0]['attachments'][0]['poll']['question']
        stats_mech[title] = [record['comments']['count'], record['likes']['count'], record['reposts']['count'], record['date']]

In [3]:
# Суммируем лайки, репосты и комменты постов хабра
sum_habr = 0
for line in stats_habr.values():
    sum_habr += (line[0]+line[1]+line[2])    

# Суммируем лайки, репосты и комменты постов механики
sum_mech = 0
for line in stats_mech.values():
    sum_mech += (line[0]+line[1]+line[2])  

In [4]:
# Получаем количество подписчиков хабра
url = 'https://api.vk.com/method/groups.getMembers'
params = {
    'group_id': 'habr',
    'v': 5.73,
    'access_token': token
}
r_habr = requests.get(url, params = params)
count_habr = r_habr.json()['response']['count']

# Получаем количество подписчиков механики
url = 'https://api.vk.com/method/groups.getMembers'
params = {
    'group_id': 'popularmechanics',
    'v': 5.73,
    'access_token': token
}
r_mech = requests.get(url, params = params)
count_mech = r_mech.json()['response']['count']

In [5]:
# Расчитываем индексы вовлеченности групп и печатаем их
involve_habr = round(count_habr / sum_habr, 1)
involve_mech = round(count_mech / sum_mech, 1)
print(involve_habr)
print(involve_mech)

112.6
199.4
